# 3 Codificando Mecanismos de atenção

* Explorando as razões para usar mecanismos de atenção em redes neurais

* Apresentando uma estrutura básica de autoatenção e progredindo para um mecanismo aprimorado de autoatenção

* Implementando um módulo de atenção causal que permite que LLMs gerem um token por vez

* Mascarando pesos de atenção selecionados aleatoriamente com abandono para reduzir o overfitting

* Empilhar vários módulos de atenção causal em um módulo de atenção com várias cabeças

<img src="https://drek4537l1klr.cloudfront.net/raschka/v-4/Figures/03__image013.png">

Considere o texto:
"Your journey starts with one step."

Aqui esses vetores são representados como tridimensionais.
O modelo GPT2 possui 768 dimensões para cada token (palavra ou subpalavra)

In [ ]:
import torch
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

O objetivo geral desta seção é ilustrar o cálculo do vetor de contexto z (2) usando a segunda sequência de entrada, x (2) como uma consulta. Esta figura mostra a primeira etapa intermediária, calculando as pontuações de atenção ω entre a consulta x (2) e todos os outros elementos de entrada como um produto escalar.

<img src="https://drek4537l1klr.cloudfront.net/raschka/v-4/Figures/03__image015.png">

In [ ]:
print(inputs.shape[0]) # quantidade de linhas
print(inputs.shape[1]) # quantidade de colunas

6
3


In [ ]:
query = inputs[1] # O segundo token de entrada serve como consulta (query)
attn_scores_2 = torch.empty(inputs.shape[0]) # cria um tensor vazio com o mesmo tamanho do numero de vetores de entrada
for i, x_i in enumerate(inputs):
  attn_scores_2[i] = torch.dot(x_i, query) # Para cada query definida calculamos o produto escalar entre esse vetor da query e cada outro vetor de entrada

print(attn_scores_2)


tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [ ]:
# entendendo o i e x_i dos inputs
for i, x_i in enumerate(inputs):
  print(i, '------->', x_i)

0 -------> tensor([0.4300, 0.1500, 0.8900])
1 -------> tensor([0.5500, 0.8700, 0.6600])
2 -------> tensor([0.5700, 0.8500, 0.6400])
3 -------> tensor([0.2200, 0.5800, 0.3300])
4 -------> tensor([0.7700, 0.2500, 0.1000])
5 -------> tensor([0.0500, 0.8000, 0.5500])


Compreendendo os Produtos Escalares

é uma maneira de multiplicarmos dois vetores elemento a elemento e depois somar os produtos

In [ ]:
res = 0.
for idx, element in enumerate(inputs[0]):
  res += inputs[0][idx] * query[idx]
print(res)
print(torch.dot(inputs[0], query))

tensor(0.9544)
tensor(0.9544)


Depois de calcular os scores de atenção W21 a W2T em relação á query de entrada x(2), o proximo passo é obter os pesos de atenção
a21 a a2T  normalizando os scores de atenção

<img src="https://drek4537l1klr.cloudfront.net/raschka/v-4/Figures/03__image017.png">

O principal objetivo por trás da normalização é obter pesos de atenção que somam 1.

In [ ]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()
print("Attention weights: ", attn_weights_2_tmp)
print("sum: ", attn_weights_2_tmp.sum())

Attention weights:  tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
sum:  tensor(1.0000)


Na prática, é mais comum e aconselhável utilizar a função softmax para normalização. Esta abordagem é melhor no gerenciamento de valores extremos e oferece propriedades de gradiente mais favoráveis ​​durante o treinamento. Abaixo está uma implementação básica da função softmax para normalizar as pontuações de atenção:

In [ ]:
def softmax_naive(x):
  return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)
print("Attention weights:", attn_weights_2_naive)
print("Sum: ", attn_weights_2_naive.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum:  tensor(1.)


Esta implementação ingenua do softmax pode trazer problemas de instabilidade numerica.

Portanto é recomendavel usar a implementação softmax do pytorch



In [ ]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0) #Uma dimensão ao longo da qual o Softmax será calculado
print("Attention weights: ", attn_weights_2)
print("Sum: ", attn_weights_2.sum())

Attention weights:  tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum:  tensor(1.)


A etapa final, após calcular e normalizar os escores de atenção para obter os pesos de atenção para a consulta x (2) , é calcular o vetor de contexto z (2) . Este vetor de contexto é uma combinação de todos os vetores de entrada x (1) a x (T) ponderados pelos pesos de atenção.

<img src="https://drek4537l1klr.cloudfront.net/raschka/v-4/Figures/03__image019.png">


In [ ]:
query = inputs[1]
context_vec_2 = torch.zeros(query.shape) # tensor de zeros, query.shape = 3
for i, x_i in enumerate(inputs):
  context_vec_2 += attn_weights_2[i] * x_i
print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


## Calculando pesos de atenção para todos os tokens de entrada

<img src="https://drek4537l1klr.cloudfront.net/raschka/v-4/Figures/03__image023.png">

In [ ]:
attn_scores = torch.empty(6,6)
for i, x_i in enumerate(inputs):
  for j, x_j in enumerate(inputs):
    attn_scores[i, j] = torch.dot(x_i, x_j)
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


Ao calcular o tensor de pontuação de atenção anterior, usamos loops for em Python. No entanto, os loops for geralmente são lentos e podemos obter os mesmos resultados usando a multiplicação de matrizes:

In [ ]:
attn_scores = inputs @ inputs.T # multiplicação de uma matriz 6x3 com outra 3x6 resultando em uma matriz 6x6
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


Na etapa 2, normalizamos cada linha para que os valores em cada linha somem 1

In [ ]:
attn_weights = torch.softmax(attn_scores, dim=1)
print(attn_weights)
print(attn_weights[0].sum())

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])
tensor(1.0000)


In [ ]:
# Vamos verificar msm
row_2_sum = sum([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
print("Row 2 sum:", row_2_sum)
print("All row sums:", attn_weights.sum(dim=1))

Row 2 sum: 1.0
All row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


Gerando essa imagem:

<img src="https://drek4537l1klr.cloudfront.net/raschka/v-4/Figures/03__image021.png">

A linha destacada mostra os pesos de atenção para o segundo elemento de entrada como uma consulta, conforme calculamos na seção anterior. Esta seção generaliza o cálculo para obter todos os outros pesos de atenção.

Na terceira e ultima etapa, usamos agora esses pesos de atenção para calcular todos os vetores de contexto por meio da multiplicação de matrizes

In [ ]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [ ]:
# verificando se o codigo está correto comparando context_vec_2 calculado anteriormente
print("Previous 2nd context vector:", context_vec_2)

Previous 2nd context vector: tensor([0.4419, 0.6515, 0.5683])


Isso conclui o passo a passo do código de um mecanismo simples de autoatenção. Na próxima seção, adicionaremos pesos treináveis, permitindo que o LLM aprenda com os dados e melhore seu desempenho em tarefas específicas.

## Implementando autoatenção com pesos treináveis

### Calculando os pesos de atenção passo a passo

<img src="https://drek4537l1klr.cloudfront.net/raschka/v-5/Figures/03__image027.png">

In [ ]:
x_2 = inputs[1] # a segunda entrada [0.5500, 0.8700, 0.6600]
d_in = inputs.shape[1] # tamanho do embedding de entrada, d = 3
d_out = 2 # tamanho do embedding de saída

In [ ]:
# Inicializando matrizes de pesos Wq, Wk e Wv
torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

In [ ]:
# Calculando os vetores query, key and value
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value
print(query_2)

tensor([0.4306, 1.4551])


Isso retorna um vetor bidimensional pois definimos o numero de colunas da matriz de pesos em `d_out=2`

Embora nosso objetivo temporário seja calcular apenas um vetor de contexto, z (2) , ainda precisamos dos vetores de chave e valor para todos os elementos de entrada, pois eles estão envolvidos no cálculo dos pesos de atenção em relação à consulta q (2) , conforme ilustrado na Figura 3.14.

Podemos obter todas as chaves e valores através da multiplicação de matrizes:

In [ ]:
keys = inputs @ W_key
values = inputs @ W_value
print("keys.shape:", keys.shape)
print("values.shape: ", values.shape)

keys.shape: torch.Size([6, 2])
values.shape:  torch.Size([6, 2])


In [ ]:
print(keys)
print("\n",values)

tensor([[0.3669, 0.7646],
        [0.4433, 1.1419],
        [0.4361, 1.1156],
        [0.2408, 0.6706],
        [0.1827, 0.3292],
        [0.3275, 0.9642]])

 tensor([[0.1855, 0.8812],
        [0.3951, 1.0037],
        [0.3879, 0.9831],
        [0.2393, 0.5493],
        [0.1492, 0.3346],
        [0.3221, 0.7863]])


O segundo passo agora é calcular os escores de atenção

**O cálculo da pontuação de atenção é um cálculo de produto escalar semelhante ao que usamos no mecanismo simplificado de autoatenção. O novo aspecto aqui é que não estamos calculando diretamente o produto escalar entre os elementos de entrada, mas usando a consulta e a chave obtidas pela transformação das entradas por meio das respectivas matrizes de pesos.**

<img src="https://drek4537l1klr.cloudfront.net/raschka/v-5/Figures/03__image029.png">


Primeiro vamos calcular a pontuação de atenção w22:

obs: esse w não é o de peso (parâmetro)


In [ ]:
keys_2 = keys[1]
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

tensor(1.8524)


In [ ]:
query_2

tensor([0.4306, 1.4551])

In [ ]:
keys.T

tensor([[0.3669, 0.4433, 0.4361, 0.2408, 0.1827, 0.3275],
        [0.7646, 1.1419, 1.1156, 0.6706, 0.3292, 0.9642]])

In [ ]:
# Generalizando o calculo para todas as pontuações(scores) de atenção
attn_scores_2 = query_2 @ keys.T # Todas as pontuações(scores) de atenção para determinado query
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


Após calcular os escores de atenção ω , o próximo passo é normalizar esses escores usando a função softmax para obter os pesos de atenção α .

<img src="https://drek4537l1klr.cloudfront.net/raschka/v-5/Figures/03__image031.png">

calculamos os pesos de atenção dimensionando as pontuações de atenção e usando a função softmax que usamos anteriormente. A diferença em relação ao anterior é que agora dimensionamos as pontuações de atenção dividindo-as pela raiz quadrada da incorporação dimensão das chaves, (observe que tirar a raiz quadrada é matematicamente o mesmo que exponenciar por 0,5):

In [ ]:
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


Na etapa final do cálculo da autoatenção, calculamos o vetor de contexto combinando todos os vetores de valor por meio dos pesos de atenção.

<img src="https://drek4537l1klr.cloudfront.net/raschka/v-5/Figures/03__image033.png">

In [ ]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.3061, 0.8210])


Até agora, calculamos apenas um unico vetor de contexto, z(2), Iremos generalizar para calcular todos os vetores de contexto na sequencia de entrada.

## Implementando uma classe Python compacta de autoatenção


In [ ]:
# Uma classe compacta de autoatenção

import torch.nn as nn
class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key   = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value
        attn_scores = queries @ keys.T # omega
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1)
        context_vec = attn_weights @ values
        return context_vec


In [ ]:
# Podemos usar a classe da seguinte forma:
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


Como uma verificação rápida, observe como a segunda linha corresponde ao conteúdo do context_vec_2 na seção anterior.


<img src="https://drek4537l1klr.cloudfront.net/raschka/v-5/Figures/03__image035.png">

Podemos melhorar o SelfAttention_v1implementação adicional utilizando PyTorch's nn.Linearcamadas, que realizam efetivamente a multiplicação de matrizes quando as unidades de polarização estão desativadas. Além disso, uma vantagem significativa de usar nn.Linearem vez de implementar manualmente nn.Parameter(torch.rand(...))é aquele nn.Linearpossui um esquema de inicialização de peso otimizado, contribuindo para um treinamento de modelo mais estável e eficaz.

In [ ]:
# classe de self-attention usando camadas lineares do pytorch

class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=1)
        context_vec = attn_weights @ values
        return context_vec

In [ ]:
torch.manual_seed(123)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.5337, -0.1051],
        [-0.5323, -0.1080],
        [-0.5323, -0.1079],
        [-0.5297, -0.1076],
        [-0.5311, -0.1066],
        [-0.5299, -0.1081]], grad_fn=<MmBackward0>)


## Escondendo palavras futuras com atenção causal

<img src="https://drek4537l1klr.cloudfront.net/raschka/v-5/Figures/03__image037.png">



Uma forma de obter a matriz de pesos de atenção mascarada na atenção causal é aplicar a função softmax aos escores de atenção, zerando os elementos acima da diagonal e normalizando a matriz resultante.
<img src="https://drek4537l1klr.cloudfront.net/raschka/v-5/Figures/03__image039.png">

In [ ]:
#calculando os pesos de atenção usando a função softmax
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)
attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=1)
print(attn_weights)

tensor([[0.1717, 0.1762, 0.1761, 0.1555, 0.1627, 0.1579],
        [0.1636, 0.1749, 0.1746, 0.1612, 0.1605, 0.1652],
        [0.1637, 0.1749, 0.1746, 0.1611, 0.1606, 0.1651],
        [0.1636, 0.1704, 0.1702, 0.1652, 0.1632, 0.1674],
        [0.1667, 0.1722, 0.1721, 0.1618, 0.1633, 0.1639],
        [0.1624, 0.1709, 0.1706, 0.1654, 0.1625, 0.1682]],
       grad_fn=<SoftmaxBackward0>)


Podemos implementar a etapa 2 usando a função `tril`do pytorch para criar uma mascara onde os valores acima da diagonal são zero

In [ ]:
# torch.tril() -> retorna a parte inferior diagonal da matriz, a parte superior é preenchida com zeros
a = torch.randn(6,6)
print(a)
s = torch.tril(a)
print("\n",s)

tensor([[-0.1690,  0.9178,  1.5810,  1.3010,  1.2753, -0.2010],
        [-0.1606, -0.4015,  0.9666, -1.1481, -1.1589,  0.3255],
        [-0.6315, -2.8400, -0.7849, -1.4096, -2.1338,  1.0524],
        [-0.3885, -0.9343,  1.0533,  0.1388, -0.2044, -2.2685],
        [-0.9133, -0.4204,  1.3111, -0.2199,  0.1838,  0.2293],
        [ 0.6177, -0.2876,  0.8218,  0.1512,  0.1036, -2.1996]])

 tensor([[-0.1690,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.1606, -0.4015,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.6315, -2.8400, -0.7849,  0.0000,  0.0000,  0.0000],
        [-0.3885, -0.9343,  1.0533,  0.1388,  0.0000,  0.0000],
        [-0.9133, -0.4204,  1.3111, -0.2199,  0.1838,  0.0000],
        [ 0.6177, -0.2876,  0.8218,  0.1512,  0.1036, -2.1996]])


In [ ]:
block_size = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(block_size, block_size))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


Multiplicando esta mascara pelos attention weights (pesos de atenção) para zerar os valores acima da diagonal

In [ ]:
masked_simple = attn_weights*mask_simple
print(masked_simple)

tensor([[0.1717, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1636, 0.1749, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1637, 0.1749, 0.1746, 0.0000, 0.0000, 0.0000],
        [0.1636, 0.1704, 0.1702, 0.1652, 0.0000, 0.0000],
        [0.1667, 0.1722, 0.1721, 0.1618, 0.1633, 0.0000],
        [0.1624, 0.1709, 0.1706, 0.1654, 0.1625, 0.1682]],
       grad_fn=<MulBackward0>)


O terceiro passo é renormalizar os pesos de atenção (attention weights) para somar 1 novamente em cada linha

In [ ]:
row_sums = masked_simple.sum(dim=1, keepdim=True) #dim=1 -> soma ao longo das dimensões das linhaas
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4833, 0.5167, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3190, 0.3408, 0.3402, 0.0000, 0.0000, 0.0000],
        [0.2445, 0.2545, 0.2542, 0.2468, 0.0000, 0.0000],
        [0.1994, 0.2060, 0.2058, 0.1935, 0.1953, 0.0000],
        [0.1624, 0.1709, 0.1706, 0.1654, 0.1625, 0.1682]],
       grad_fn=<DivBackward0>)


In [ ]:
row_sums

tensor([[0.1717],
        [0.3385],
        [0.5132],
        [0.6693],
        [0.8361],
        [1.0000]], grad_fn=<SumBackward1>)

**Uma maneira mais eficiente de obter a matriz de pesos de atenção mascarada na atenção causal é mascarar os scores de atenção com valores infinitos negativos antes de aplicar a função softmax.**

<img src="https://drek4537l1klr.cloudfront.net/raschka/v-5/Figures/03__image041.png">

A função softmax converte suas entradas em uma distribuição de probabilidade. Quando valores infinitos negativos (-∞) estão presentes em uma linha, a função softmax os trata como probabilidade zero. (Matematicamente, isso ocorre porque e - ∞ se aproxima de 0.)

Podemos implementar esse "truque" de mascaramento mais eficiente criando uma máscara com 1's acima da diagonal e então substituindo esses 1's por infinito negativo ( -inf) valores:

In [ ]:
# torch.triu retorna valores de 0 na diagonal inferior

In [ ]:
mask = torch.triu(torch.ones(block_size, block_size), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[0.3111,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.1655, 0.2602,   -inf,   -inf,   -inf,   -inf],
        [0.1667, 0.2602, 0.2577,   -inf,   -inf,   -inf],
        [0.0510, 0.1080, 0.1064, 0.0643,   -inf,   -inf],
        [0.1415, 0.1875, 0.1863, 0.0987, 0.1121,   -inf],
        [0.0476, 0.1192, 0.1171, 0.0731, 0.0477, 0.0966]],
       grad_fn=<MaskedFillBackward0>)


Agora tudo o que precisamos fazer é aplicar a função softmax a esses resultados mascarados e pronto:

In [ ]:
attn_weights = torch.softmax(masked / keys.shape[-1] ** 0.5, dim=1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4833, 0.5167, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3190, 0.3408, 0.3402, 0.0000, 0.0000, 0.0000],
        [0.2445, 0.2545, 0.2542, 0.2468, 0.0000, 0.0000],
        [0.1994, 0.2060, 0.2058, 0.1935, 0.1953, 0.0000],
        [0.1624, 0.1709, 0.1706, 0.1654, 0.1625, 0.1682]],
       grad_fn=<SoftmaxBackward0>)


Como podemos ver com base na saída, os valores em cada linha somam 1 e nenhuma normalização adicional é necessária

Poderíamos agora usar os pesos de atenção modificados para calcular os vetores de contexto via context_vec = attn_weights @ values, como na seção 3.4. No entanto, na próxima seção, abordaremos primeiro outro pequeno ajuste no mecanismo de atenção causal que é útil para reduzir o overfitting ao treinar LLMs.

##  Mascarando pesos de atenção adicionais com dropout

<img src="https://drek4537l1klr.cloudfront.net/raschka/v-5/Figures/03__image043.png">

In [ ]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5) #dropout de 50%
example = torch.ones(6, 6) #matriz de 1's
print(dropout(example))

tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])


Ao aplicar o dropout a uma matriz de peso de atenção com uma taxa de 50%, metade dos elementos da matriz são definidos aleatoriamente como zero.  Para compensar a redução nos elementos ativos, os valores dos elementos restantes na matriz são aumentados por um fator de 1/0,5 =2.

In [ ]:
# aplicando o dropout a matriz de peso
torch.manual_seed(123)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.0335, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6804, 0.0000, 0.0000, 0.0000],
        [0.4889, 0.5090, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3988, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3418, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)


## Implementando uma classe compacta de atenção causal

Mas antes de começarmos, mais uma coisa é garantir que o código possa lidar com lotes que consistem em mais de uma entrada, para que o CausalAttentionclasse suporta as saídas em lote produzidas pelo carregador de dados que implementamos no capítulo 2.

Para simplificar, para simular tais entradas em lote, duplicamos o exemplo do texto de entrada:

In [ ]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape)

#2 entradas com 6 tokens cada, e cada token tem dimensão de incorporação 3

torch.Size([2, 6, 3])


In [ ]:
batch

tensor([[[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]],

        [[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]]])

In [ ]:
 # Uma classe compacta de atenção causal

class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, block_size, dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
           'mask',
           torch.triu(torch.ones(block_size, block_size),
           diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        #New batch dimension b
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2)
        attn_scores.masked_fill_(
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=1)
        attn_weights = self.dropout(attn_weights)

        context_vec = attn_weights @ values
        return context_vec

In [ ]:
torch.manual_seed(123)
block_size = batch.shape[1]
ca = CausalAttention(d_in, d_out, block_size, 0.0)
context_vecs = ca(batch)
print("context_vecs.shape:", context_vecs.shape)



context_vecs.shape: torch.Size([2, 6, 2])


## Estendendo a atenção de uma única cabeça para a atenção de múltiplas cabeças (multi-head)

O módulo de atenção com múltiplas cabeças nesta figura representa dois módulos de atenção com uma única cabeça empilhados um sobre o outro. Portanto, em vez de usar uma única matriz W v para calcular as matrizes de valor, em um módulo de atenção multicabeças com duas cabeças, agora temos duas matrizes de peso de valor: W v1 e W v2 . O mesmo se aplica às outras matrizes de peso, W q e W k . Obtemos dois conjuntos de vetores de contexto Z 1 e Z 2 que podemos combinar em uma única matriz de vetor de contexto Z .

<img src="https://drek4537l1klr.cloudfront.net/raschka/v-5/Figures/03__image047.png">

In [ ]:
# Uma classe wrapper para implementar atenção multi-head

class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, block_size,
                 dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, block_size, dropout, qkv_bias)
             for _ in range(num_heads)]
        )

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)


Usando o MultiHeadAttentionWrapper, especificamos o número de cabeças de atenção ( num_heads). Se definirmos num_heads=2, conforme mostrado nesta figura, obtemos um tensor com dois conjuntos de matrizes de vetores de contexto. Em cada matriz de vetor de contexto, as linhas representam os vetores de contexto correspondentes aos tokens e as colunas correspondem à dimensão de incorporação especificada via d_out=2. Concatenamos essas matrizes de vetores de contexto ao longo da dimensão da coluna. Como temos 2 cabeças de atenção e uma dimensão de incorporação de 4, a dimensão de incorporação final é 2 × 2 = 4.

<img src="https://drek4537l1klr.cloudfront.net/raschka/v-5/Figures/03__image049.png">

In [ ]:
# Usando a classe
torch.manual_seed(123)
block_size = batch.shape[1] # numero de tokens
d_in, d_out = 3, 2

mha = MultiHeadAttentionWrapper(d_in, d_out, block_size, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.0844,  0.0414,  0.0766,  0.0171],
         [-0.2264, -0.0039,  0.2143,  0.1185],
         [-0.4163, -0.0564,  0.3878,  0.2453],
         [-0.5014, -0.1011,  0.4992,  0.3401],
         [-0.7754, -0.1867,  0.7387,  0.4868],
         [-1.1632, -0.3303,  1.1224,  0.8460]],

        [[-0.0844,  0.0414,  0.0766,  0.0171],
         [-0.2264, -0.0039,  0.2143,  0.1185],
         [-0.4163, -0.0564,  0.3878,  0.2453],
         [-0.5014, -0.1011,  0.4992,  0.3401],
         [-0.7754, -0.1867,  0.7387,  0.4868],
         [-1.1632, -0.3303,  1.1224,  0.8460]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


## Implementando atenção multicabeça com divisão de peso

Em vez de manter duas classes separadas, MultiHeadAttentionWrappere CausalAttention, podemos combinar esses dois conceitos em uma única classe MultiHeadAttention.



o seguinte MultiHeadAttentionclass integra a funcionalidade multi-head em uma única classe. Ele divide a entrada em vários heads, remodelando a consulta projetada, a chave e os tensores de valor e, em seguida, combina os resultados desses heads após calcular a atenção.

In [ ]:
# Uma classe eficiente de multi-head attention
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out,
                 block_size, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by n_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out) # combinar as saídas das heads
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            'mask',
             torch.triu(torch.ones(block_size, block_size), diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        attn_scores = queries @ keys.transpose(2, 3)
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        mask_unsqueezed = mask_bool.unsqueeze(0).unsqueeze(0)
        attn_scores.masked_fill_(mask_unsqueezed, -torch.inf)

        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        context_vec = (attn_weights @ values).transpose(1, 2)

        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)
        return context_vec



 No MultiheadAttentionWrapperclasse com duas cabeças de atenção, inicializamos duas matrizes de peso W q1 e W q2 e calculamos duas matrizes de consulta Q 1 e Q 2 conforme ilustrado na parte superior desta figura. No MultiheadAttentionclasse, inicializamos uma matriz de peso maior W q , realizamos apenas uma multiplicação de matriz com as entradas para obter uma matriz de consulta Q e, em seguida, dividimos a matriz de consulta em Q 1 e Q 2 conforme mostrado na parte inferior desta figura. Fazemos o mesmo com as chaves e valores, que não são mostrados para reduzir a confusão visual.

 <img src="https://drek4537l1klr.cloudfront.net/raschka/v-5/Figures/03__image051.png">

Para ilustrar esta multiplicação de matrizes em lote, suponha que temos o seguinte exemplo de tensor:



In [ ]:
a = torch.tensor([[[[0.2745, 0.6584, 0.2775, 0.8573],
                    [0.8993, 0.0390, 0.9268, 0.7388],
                    [0.7179, 0.7058, 0.9156, 0.4340]],

                   [[0.0772, 0.3565, 0.1479, 0.5331],
                    [0.4066, 0.2318, 0.4545, 0.9737],
                    [0.4606, 0.5159, 0.4220, 0.5786]]]])

In [ ]:
a.shape # b, num_heads, num_tokens, head_dim

torch.Size([1, 2, 3, 4])

Agora, realizamos uma multiplicação de matrizes em lote entre o próprio tensor e uma visão do tensor onde transpomos as duas últimas dimensões, num_tokense head_dim:

In [ ]:
print(a @ a.transpose(2, 3))

tensor([[[[1.3208, 1.1631, 1.2879],
          [1.1631, 2.2150, 1.8424],
          [1.2879, 1.8424, 2.0402]],

         [[0.4391, 0.7003, 0.5903],
          [0.7003, 1.3737, 1.0620],
          [0.5903, 1.0620, 0.9912]]]])


In [ ]:
first_head = a[0, 0, :, :]
first_res = first_head @ first_head.T
print("First head:\n", first_res)

second_head = a[0, 1, :, :]
second_res = second_head @ second_head.T
print("\nSecond head:\n", second_res)

First head:
 tensor([[1.3208, 1.1631, 1.2879],
        [1.1631, 2.2150, 1.8424],
        [1.2879, 1.8424, 2.0402]])

Second head:
 tensor([[0.4391, 0.7003, 0.5903],
        [0.7003, 1.3737, 1.0620],
        [0.5903, 1.0620, 0.9912]])


In [ ]:
# Usando essa bodega

torch.manual_seed(123)
batch_size, block_size, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, block_size, 0.0, num_heads=2)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


## **register_buffer()**

Em PyTorch, buffers são tensores que são armazenados como parte do estado de um módulo, mas não são atualizados durante o treinamento. Isso significa que eles não são otimizados pelo otimizador e seus valores não mudam durante a propagação para trás. Buffers são úteis para armazenar dados que precisam ser persistidos com o modelo, mas que não precisam ser atualizados durante o treinamento.

**Diferença entre Buffers e Parâmetros**

A principal diferença entre buffers e parâmetros é que os parâmetros são atualizados durante o treinamento, enquanto os buffers não. Parâmetros são tipicamente os pesos de uma rede neural, enquanto buffers podem ser usados para armazenar outros tipos de dados, como contagens de iteração, estatísticas de normalização ou embeddings.

In [ ]:
import torch

class MyModule(nn.Module):
  def __init__(self):
    super().__init__()
    self.register_buffer("counter", torch.ones(2, 3))

module = MyModule()
print(module.counter)

tensor([[1., 1., 1.],
        [1., 1., 1.]])


In [ ]:
class MyModule(nn.Module):
  def __init__(self):
    super().__init__()
    self.register_buffer('counter', torch.ones(6,6))

module = MyModule()
print(module.counter)

tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]])


## **torch.view()**

O método .view() em PyTorch é usado para alterar a forma de um tensor sem copiar seus dados subjacentes. Isso significa que o tensor original e o tensor retornado pelo .view() compartilham os mesmos dados. O método .view() é uma ferramenta poderosa que pode ser usada para realizar uma variedade de operações em tensores, como transposição, achatamento e reshape.

In [ ]:
# achatamento

x = torch.tensor([[1,2,3], [4,5,6]])
print(x)

y = x.view(3,2)
print('\n',y)

tensor([[1, 2, 3],
        [4, 5, 6]])

 tensor([[1, 2],
        [3, 4],
        [5, 6]])


In [ ]:
# Redimensionamento com Inferência de dimensões

x = torch.arange(10)
print(x)

x_reshaped = x.view(-1, 5)
print("\n",x_reshaped)

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

 tensor([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])


In [ ]:
# achatamento

x = torch.tensor([[1,2,3], [4,5,6]])
print(x)

y = x.view(-1)
print("\n", y)

tensor([[1, 2, 3],
        [4, 5, 6]])

 tensor([1, 2, 3, 4, 5, 6])
